## Immutable Dataclass
`` The instance cannot be modified after creation. It means the field will not be modified when an instance is created ``

In [16]:
"hello world"

'hello world'

In [17]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Point:
    x : float
    y : float


p = Point(x=12 , y=20)

print(p)

try:
    p.x = 90
except Exception as e:
    print(e , "Because it is immutable")




Point(x=12, y=20)
cannot assign to field 'x' Because it is immutable


In [18]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Rectangle:
    top_left : Point
    bottom_right : Point



    @property
    def width(self) -> float:
        return (self.top_left.x - self.bottom_right.x)
    
    @property
    def height(self) -> float:
        return (self.top_left.y - self.bottom_right.y)

    @property
    def area(self) -> float:
        return (self.width * self.height)
    



In [35]:
from typing import Tuple , FrozenSet
from dataclasses import field , dataclass

@dataclass(frozen=True)
class ImmutableConfig:
    """Configuration that cannot be changed after creation."""
    app_name: str
    version: str
    max_connections: int
    # Using immutable types for collections
    allowed_paths: Tuple[str, ...] = field(default_factory=tuple)
    feature_flags: FrozenSet[str] = field(default_factory=frozenset)
    
    def __post_init__(self):
        """Validate the configuration after initialization."""
        # Since we can't modify self directly in a frozen dataclass, this only validates
        if not self.app_name:
            raise ValueError("App name cannot be empty")
        if self.max_connections <= 0:
            raise ValueError("Max connections must be positive")

In [38]:
if __name__ == "__main__":
    
    p1 = Point(2 , 3)
    p2 = Point(5 , 8)
    print(p1)
    print(p2)

    rec = Rectangle(p1 , p2)
    print(rec)
    print(f"Area of rectanglge with the given width of {rec.width} and given height of {rec.height} = " ,rec.area)

    try:
        rec.top_left = Point(10 , 20)
    except Exception as e:
        print("Cannot be modified because of immuatable")

    #create an immutable configuration
    config = ImmutableConfig(
        app_name="MyApp",
        version="1.0.0",
        max_connections=100,
        allowed_paths=("/api", "/public", "/assets"),
        feature_flags=frozenset({"dark_mode", "new_ui"})
    )
    
    print(f"Config: {config}")


Point(x=2, y=3)
Point(x=5, y=8)
Rectangle(top_left=Point(x=2, y=3), bottom_right=Point(x=5, y=8))
Area of rectanglge with the given width of -3 and given height of -5 =  15
Cannot be modified because of immuatable
Config: ImmutableConfig(app_name='MyApp', version='1.0.0', max_connections=100, allowed_paths=('/api', '/public', '/assets'), feature_flags=frozenset({'dark_mode', 'new_ui'}))


In [41]:
# BAD EXAMPLE 1: Mutable attributes in immutable dataclass
from typing import Dict , List 

@dataclass(frozen=True)
class BadImmutable:
    name: str
    # A mutable list in an "immutable" class
    values: List[int] = field(default_factory=list)
    # A mutable dict in an "immutable" class
    metadata: Dict[str, str] = field(default_factory=dict)


def demo_bad_immutable():
    # Create a supposedly immutable object
    bad = BadImmutable("example", [1, 2, 3], {"key": "value"})
    
    print("\n=== BAD IMMUTABLE DATACLASS EXAMPLE ===")
    print(f"Original: {bad}")
    
    # The object itself is immutable (can't reassign attributes)
    try:
        bad.name = "new_name"
    except Exception as e:
        print(f"Cannot modify attribute: {e}")
    
    # But you can modify the mutable contents!
    bad.values.append(4)
    bad.metadata["new_key"] = "new_value"
    
    print(f"After modifications to internal mutable objects: {bad}")
    print("This breaks immutability expectations!")


# BAD EXAMPLE 2: Trying to use __init__ to modify a frozen dataclass
@dataclass(frozen=True)
class BadInitImmutable:
    value: int
    squared: int = None  # We want to compute this in init
    
    def __init__(self, value: int):
        # This will fail!
        self.value = value
        self.squared = value ** 2


# FIXED EXAMPLE: Using __post_init__ and object.__setattr__ for initialization
@dataclass(frozen=True)
class GoodInitImmutable:
    value: int
    squared: int = field(init=False)  # Don't include in init parameters
    
    def __post_init__(self):
        # Use object.__setattr__ to bypass the frozen restriction during initialization
        object.__setattr__(self, "squared", self.value ** 2)


def demo_init_issues():
    print("\n=== ISSUES WITH INITIALIZING FROZEN DATACLASSES ===")
    
    # This will fail
    try:
        bad_init = BadInitImmutable(5)
        print(f"BadInitImmutable: {bad_init}")
    except Exception as e:
        print(f"BadInitImmutable error: {e}")
    
    # This works correctly
    good_init = GoodInitImmutable(5)
    print(f"GoodInitImmutable: {good_init}")


if __name__ == "__main__":
    print("=== GOOD IMMUTABLE DATACLASS EXAMPLES ===")
    
    print("\n=== BAD IMMUTABLE DATACLASS EXAMPLES ===")
    demo_bad_immutable()
    
    demo_init_issues() 

=== GOOD IMMUTABLE DATACLASS EXAMPLES ===

=== BAD IMMUTABLE DATACLASS EXAMPLES ===

=== BAD IMMUTABLE DATACLASS EXAMPLE ===
Original: BadImmutable(name='example', values=[1, 2, 3], metadata={'key': 'value'})
Cannot modify attribute: cannot assign to field 'name'
After modifications to internal mutable objects: BadImmutable(name='example', values=[1, 2, 3, 4], metadata={'key': 'value', 'new_key': 'new_value'})
This breaks immutability expectations!

=== ISSUES WITH INITIALIZING FROZEN DATACLASSES ===
BadInitImmutable error: cannot assign to field 'value'
GoodInitImmutable: GoodInitImmutable(value=5, squared=25)
